# Homework 2 Keys

### DUE: 02/16/2017 before class at 10:30am

The objective of this homework is for gaining experiences in designing *streaming* algorithms. Similar to Homework 1, we’re going to use the Citibike data sets posted on NYU Classes. In addition to **citibike.csv** we will also use **citibike_docking_events.csv**. Both of the data sets can be downloaded from the NYU Classes's Resources page. You are required to turn in this notebook with all the parts filled in place of <###>. Your notebook must be named BDM\_HW2\_**NetId**.ipynb.



## Task 1 (6 points)

Your task is to **compute the maximum number of active "citibikers"** that were using the Citibike service at any point in time. This the same as computing the maximum number of citibikes that were checked out at a particular time. The input data set is **citibike_docking_events.csv**, which logged all docking and undocking events at all Citibike stations. The description of the fields in this file is as follows:

|Column name|Description|
|--|--|
|time |The timestamp of the event. All events are sorted increasingly by their timestamps. |
|bikeid |The unique ID of the bike involved in this event. |
|station_id |The station ID, where the event happened. |
|event |A string of either *"dock"* or *"undock"* for describing the drop-off or pick-up event, respectively. |

For example, let's assume that on *Feb-01-2015*, there was a user that picked a bike at the station ID *521* at midnight and dropped it at the station ID *423* at 14 minutes past midnight. If the bike that this customer used has the ID of *17131*, then you should see two events being logged in this data set as:

<pre>
...
2015-02-01 00:00:00+00,17131,521,undock
...
2015-02-01 00:14:00+00,17131,423,dock
...
</pre>

You are given the above data set in a streaming fashion (reading in row by row), and must design a streaming algorithm that uses the least possible additional memory to compute the maximum number of active users of the Citibike service. Please modify the code snippet below to complete this task. Your code should only output a single number, which is the number of active users. Of course, you can add additional initialization codes outside of the for loop as needed.
Additional, please provide a brief rationale and/or justification for your design after the code.

In [2]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

activeUsers = 0
maxActiveUsers,maxTime = 0,None
for row in csvRows('citibike_docking_events.csv'):
    if row['event']=='undock':
        activeUsers += 1
        if activeUsers>maxActiveUsers:
            maxActiveUsers,maxTime = activeUsers,row['time']
    else:
        activeUsers -= 1
    
print maxActiveUsers
print maxTime

250
2015-02-05 08:55:00+00


#### RATIONALE AND JUSTIFICATION

*Please explain your solution here*

## Task 2 (4 points)

The objective of this task is identical to Task 1's but you are asked to use the **cibibike.csv** data set instead of the docking events. The main difference (and challenge) is that both pick-up and drop-off event for each trip is now presented as a single record, thus, the drop-off events are not sorted by their timestamps. You are again asked to do this in a streaming fashion that needs to minimize the amount of memory usage. Please modify the code below accordingly, and also with a brief explaination of the solution.

In [20]:
import csv

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

notYetReturn = []
maxActiveUsers,maxTime = 0,None
for row in csvRows('citibike.csv'):
    starttime = row['starttime']
    stoptime  = row['stoptime']
    notYetReturn = filter(lambda x: x>starttime, notYetReturn) + [stoptime]
    if len(notYetReturn)>maxActiveUsers:
        maxActiveUsers,maxTime = len(notYetReturn),starttime
    
print maxActiveUsers
print maxTime

250
2015-02-05 08:55:00+00


In [17]:
import csv
import heapq

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row

notYetReturn = []
maxActiveUsers,maxTime = 0,None
for row in csvRows('citibike.csv'):
    starttime = row['starttime']
    stoptime  = row['stoptime']
    heapq.heappush(notYetReturn, stoptime)
    while starttime>=notYetReturn[0]:
        heapq.heappop(notYetReturn)
    if len(notYetReturn)>maxActiveUsers:
        maxActiveUsers,maxTime = len(notYetReturn),starttime
    
print maxActiveUsers
print maxTime

250
2015-02-05 08:55:00+00


In [18]:
import csv
import heapq

def csvRows(filename):
    with open(filename, 'r') as fi:
        reader = csv.DictReader(fi)
        for row in reader:
            yield row
            
def createEvents(rows):
    notYetReturn = []
    for row in rows:
        starttime = row['starttime']
        stoptime  = row['stoptime']
        heapq.heappush(notYetReturn, stoptime)
        while starttime>=notYetReturn[0]:
            yield {'time': heapq.heappop(notYetReturn), 'event': 'dock'}
        yield {'time': starttime, 'event': 'undock'}

activeUsers = 0
maxActiveUsers,maxTime = 0,None
for row in createEvents(csvRows('citibike.csv')):
    if row['event']=='undock':
        activeUsers += 1
        if activeUsers>maxActiveUsers:
            maxActiveUsers,maxTime = activeUsers,row['time']
    else:
        activeUsers -= 1

print maxActiveUsers
print maxTime

250
2015-02-05 08:55:00+00


#### RATIONALE AND JUSTIFICATION

*Please explain your solution here*